In [1]:
import pandas as pd
import lmfit
import datetime
from datetime import date
from sklearn import linear_model
import datetime
import argparse
# from models.get_all_data import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from file_struct import locate_data_
from models.get_all_data import *

In [5]:
weir='211L_211K'
data_path = '../data/feature_tables/' #(--data_path) #(--weir)
risk_date='2021-07-16' # (--risk_date)
prediction=True # True for prediction (--prediction)
last_days=7 # (--last_days) For prediction: Defines how many days the linear model takes into account to predict the next 21 days
avg_temp=22 # (--avg_temp) For prediction: Average Temperature adjusts the prediction +/- 20%

In [6]:
def get_predictions(min_date:str, max_date:str, weir, data_path, path_to_full_weather = '../data/full_weather.csv'):

    full_weather = pd.read_csv(path_to_full_weather, header = 0)
    full_weather['Datum'] = full_weather['Date']
    full_weather['Date'] = pd.to_datetime(full_weather['Date'])
    full_weather = full_weather.set_index('Date')
    full_weather = full_weather[min_date: max_date]
    df = pd.DataFrame()
    df_vegetation = pd.DataFrame()
    list_of_pred_dict = []
    for i in range(len(full_weather)):
        df_21, last_data_raw = predict_vegetation(
            weir, last_days, full_weather["Avg temp"][i], data_path, risk_date=full_weather['Datum'][i])
        dict_pred = {"time" : full_weather['Datum'][i], "21_day_prediction" : df_21["Predicted backwater by vegetation"].to_list()   }
        df_vegetation = df_vegetation.append(last_data_raw.loc[0])
        
        list_of_pred_dict.append(dict_pred)
        
        list_21 = df_21.values.tolist()
        df[full_weather['Datum'][i]] = list_21
    for column in df.columns:
        for i in range(len(df)):
            df[column][i] = df[column][i][0]
    df = df.transpose()

    return df, df_vegetation

In [8]:
preds2, list_of_pred = get_predictions('2020-5-1','2020-5-20',weir, data_path)
print(weir)
preds2


211L_211K


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
2020-05-01,0.130811,0.134895,0.138980,0.143065,0.147150,0.151235,0.155319,0.159404,0.163489,0.167574,...,0.175743,0.179828,0.183913,0.187998,0.192083,0.196168,0.200252,0.204337,0.208422,0.212507
2020-05-02,0.134990,0.139113,0.143236,0.147359,0.151482,0.155605,0.159728,0.163851,0.167974,0.172097,...,0.180342,0.184465,0.188588,0.192711,0.196834,0.200957,0.205080,0.209203,0.213326,0.217449
2020-05-03,0.138149,0.141951,0.145754,0.149557,0.153359,0.157162,0.160965,0.164768,0.168570,0.172373,...,0.179978,0.183781,0.187584,0.191386,0.195189,0.198992,0.202794,0.206597,0.210400,0.214202
2020-05-04,0.138893,0.141999,0.145105,0.148211,0.151317,0.154422,0.157528,0.160634,0.163740,0.166846,...,0.173058,0.176164,0.179270,0.182376,0.185482,0.188588,0.191694,0.194800,0.197906,0.201012
2020-05-05,0.144359,0.147883,0.151407,0.154930,0.158454,0.161977,0.165501,0.169024,0.172548,0.176071,...,0.183118,0.186642,0.190165,0.193689,0.197212,0.200736,0.204259,0.207783,0.211306,0.214830
2020-05-06,0.146376,0.149807,0.153237,0.156667,0.160098,0.163528,0.166958,0.170389,0.173819,0.177249,...,0.184110,0.187541,0.190971,0.194401,0.197832,0.201262,0.204692,0.208123,0.211553,0.214984
2020-05-07,0.136630,0.137494,0.138359,0.139224,0.140088,0.140953,0.141817,0.142682,0.143546,0.144411,...,0.146140,0.147005,0.147869,0.148734,0.149599,0.150463,0.151328,0.152192,0.153057,0.153922
2020-05-08,0.140551,0.141639,0.142727,0.143815,0.144904,0.145992,0.147080,0.148168,0.149256,0.150344,...,0.152520,0.153608,0.154697,0.155785,0.156873,0.157961,0.159049,0.160137,0.161225,0.162313
2020-05-09,0.147738,0.149832,0.151926,0.154020,0.156114,0.158209,0.160303,0.162397,0.164491,0.166585,...,0.170773,0.172867,0.174961,0.177055,0.179149,0.181243,0.183337,0.185432,0.187526,0.189620
2020-05-10,0.153875,0.156846,0.159816,0.162787,0.165757,0.168728,0.171698,0.174669,0.177640,0.180610,...,0.186551,0.189522,0.192492,0.195463,0.198434,0.201404,0.204375,0.207345,0.210316,0.213286


In [48]:
preds, lst = get_predictions('2020-7-1','2020-7-20',weir, data_path)
preds = preds.reset_index()
days_22 = datetime.timedelta(22)
preds['index'] = pd.to_datetime(preds["index"])


In [16]:
preds

,index,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,2020-07-01,0.297177,0.308267,0.319357,0.330447,0.341537,0.352627,0.363717,0.374806,0.385896,...,0.419166,0.430256,0.441346,0.452435,0.463525,0.474615,0.485705,0.496795,0.507885,0.518975
1,2020-07-02,0.307551,0.318337,0.329122,0.339907,0.350693,0.361478,0.372264,0.383049,0.393835,...,0.426191,0.436976,0.447762,0.458547,0.469332,0.480118,0.490903,0.501689,0.512474,0.523260
2,2020-07-03,0.312523,0.321979,0.331436,0.340892,0.350349,0.359805,0.369262,0.378719,0.388175,...,0.416545,0.426001,0.435458,0.444914,0.454371,0.463828,0.473284,0.482741,0.492197,0.501654
3,2020-07-04,0.318192,0.327176,0.336161,0.345146,0.354131,0.363116,0.372101,0.381085,0.390070,...,0.417025,0.426010,0.434994,0.443979,0.452964,0.461949,0.470934,0.479919,0.488903,0.497888
4,2020-07-05,0.322825,0.330927,0.339030,0.347132,0.355235,0.363337,0.371439,0.379542,0.387644,...,0.411951,0.420054,0.428156,0.436258,0.444361,0.452463,0.460566,0.468668,0.476770,0.484873
5,2020-07-06,0.274898,0.272243,0.269588,0.266933,0.264279,0.261624,0.258969,0.256314,0.253660,...,0.245695,0.243041,0.240386,0.237731,0.235076,0.232422,0.229767,0.227112,0.224458,0.221803
6,2020-07-07,0.201161,0.183655,0.166149,0.148643,0.131137,0.113631,0.096125,0.078619,0.061113,...,0.008595,-0.008911,-0.026417,-0.043922,-0.061428,-0.078934,-0.096440,-0.113946,-0.131452,-0.148958
7,2020-07-08,0.147014,0.120476,0.093937,0.067398,0.040860,0.014321,-0.012218,-0.038757,-0.065295,...,-0.144911,-0.171450,-0.197989,-0.224527,-0.251066,-0.277605,-0.304143,-0.330682,-0.357221,-0.383760
8,2020-07-09,0.115531,0.085889,0.056247,0.026605,-0.003037,-0.032679,-0.062321,-0.091963,-0.121605,...,-0.210531,-0.240173,-0.269815,-0.299457,-0.329099,-0.358741,-0.388383,-0.418025,-0.447667,-0.477309
9,2020-07-10,0.106607,0.079426,0.052245,0.025063,-0.002118,-0.029299,-0.056481,-0.083662,-0.110843,...,-0.192387,-0.219568,-0.246749,-0.273931,-0.301112,-0.328293,-0.355474,-0.382656,-0.409837,-0.437018


In [49]:
for i in range(1, 22):
    preds.loc[preds.shape[0]] = [preds.iloc[-1:]['index'].item() + datetime.timedelta(1)] + 21*[0]
preds

for i in range(1):
    

,index,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,2020-07-01,0.297177,0.308267,0.319357,0.330447,0.341537,0.352627,0.363717,0.374806,0.385896,...,0.419166,0.430256,0.441346,0.452435,0.463525,0.474615,0.485705,0.496795,0.507885,0.518975
1,2020-07-02,0.307551,0.318337,0.329122,0.339907,0.350693,0.361478,0.372264,0.383049,0.393835,...,0.426191,0.436976,0.447762,0.458547,0.469332,0.480118,0.490903,0.501689,0.512474,0.523260
2,2020-07-03,0.312523,0.321979,0.331436,0.340892,0.350349,0.359805,0.369262,0.378719,0.388175,...,0.416545,0.426001,0.435458,0.444914,0.454371,0.463828,0.473284,0.482741,0.492197,0.501654
3,2020-07-04,0.318192,0.327176,0.336161,0.345146,0.354131,0.363116,0.372101,0.381085,0.390070,...,0.417025,0.426010,0.434994,0.443979,0.452964,0.461949,0.470934,0.479919,0.488903,0.497888
4,2020-07-05,0.322825,0.330927,0.339030,0.347132,0.355235,0.363337,0.371439,0.379542,0.387644,...,0.411951,0.420054,0.428156,0.436258,0.444361,0.452463,0.460566,0.468668,0.476770,0.484873
5,2020-07-06,0.274898,0.272243,0.269588,0.266933,0.264279,0.261624,0.258969,0.256314,0.253660,...,0.245695,0.243041,0.240386,0.237731,0.235076,0.232422,0.229767,0.227112,0.224458,0.221803
6,2020-07-07,0.201161,0.183655,0.166149,0.148643,0.131137,0.113631,0.096125,0.078619,0.061113,...,0.008595,-0.008911,-0.026417,-0.043922,-0.061428,-0.078934,-0.096440,-0.113946,-0.131452,-0.148958
7,2020-07-08,0.147014,0.120476,0.093937,0.067398,0.040860,0.014321,-0.012218,-0.038757,-0.065295,...,-0.144911,-0.171450,-0.197989,-0.224527,-0.251066,-0.277605,-0.304143,-0.330682,-0.357221,-0.383760
8,2020-07-09,0.115531,0.085889,0.056247,0.026605,-0.003037,-0.032679,-0.062321,-0.091963,-0.121605,...,-0.210531,-0.240173,-0.269815,-0.299457,-0.329099,-0.358741,-0.388383,-0.418025,-0.447667,-0.477309
9,2020-07-10,0.106607,0.079426,0.052245,0.025063,-0.002118,-0.029299,-0.056481,-0.083662,-0.110843,...,-0.192387,-0.219568,-0.246749,-0.273931,-0.301112,-0.328293,-0.355474,-0.382656,-0.409837,-0.437018


In [10]:
def plot_graphs_last_pred(min_date, max_date, weir, data_path, path_to_full_weather = '../data/full_weather.csv'):
    min_date = datetime.datetime.strptime(min_date, "%Y-%m-%d")
    days_7 = datetime.timedelta(7)
    new_date = min_date - days_7
    new_date = new_date.strftime("%Y/%m/%d %H:%M:%S")
    new_date = new_date.replace("/", "-")

    max_date = datetime.datetime.strptime(max_date, "%Y-%m-%d")
    days_22 = datetime.timedelta(22)
    new_max_date = max_date + days_22
    new_max_date = new_max_date.strftime("%Y/%m/%d %H:%M:%S")
    new_max_date = new_max_date.replace("/", "-") 

    data = get_data(weir=weir, data_path=data_path)
    data.reset_index(inplace=True)
    data['INDEX'] = data['TIME'].copy()
    data = data.set_index('INDEX')
    # Gets lasts days but based on the custom risk_date
    data = data.loc[min_date: new_max_date]
    data["VERSCHIL"] = negative_backwater_to_zero(data["VERSCHIL"])

    df, df_vegetation = get_predictions(min_date, max_date, weir, data_path, path_to_full_weather)
    df_deri_20 = df[20]
    df_deri_20 =df_deri_20.reset_index()
    df_deri_20['index'] = pd.to_datetime(df_deri_20["index"])
    df_deri_20["index"] = df_deri_20["index"] + days_22

    df_deri_7 = df[7]
    df_deri_7 =df_deri_7.reset_index()
    df_deri_7['index'] = pd.to_datetime(df_deri_7["index"])
    df_deri_7["index"] = df_deri_7["index"] + days_7


    fig = go.Figure()
    fig.add_trace(go.Scatter(x= data["TIME"], y = data["VERSCHIL"]))#, row = counter, col = 1)
    # fig.add_trace(go.Scatter(x= data["TIME"], y = data["Q"]))#, row = counter, col = 1)
    fig.add_trace(go.Scatter(x = df_deri_7["index"], y = df_deri_7[7]))
    fig.add_trace(go.Scatter(x = df_deri_20["index"], y = df_deri_20[20]))

    fig.add_trace(go.Scatter(x = df_vegetation["INDEX"], y = df_vegetation["vegetation"]))
    # fig.add_trace(go.Scatter(x =df_day["time"], y =df_day["values"]))#, row = counter, col = 1)
    # fig.add_vline(x=last_day)#, row = counter, col = 1)
    # fig.add_vline(x=first_day)#, row = counter, col = 1)
    # fig.update_layout(title = "Day {} predictions".format(counter))
    # counter += 1

    return fig.show()

In [39]:
plot_graphs_last_pred('2020-5-1','2020-5-20',weir, data_path)

C:\Users\20193216\.conda\envs\DataScie\lib\site-packages\pandas\core\indexes\base.py:5277: FutureWarning:

Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.



In [38]:
plot_graphs_last_pred('2020-5-1','2020-6-20',weir, data_path)

C:\Users\20193216\.conda\envs\DataScie\lib\site-packages\pandas\core\indexes\base.py:5277: FutureWarning:

Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.



In [28]:
def plot_graphs(min_date, max_date, weir, data_path, path_to_full_weather = '../data/full_weather.csv'):

    min_date = datetime.datetime.strptime(min_date, "%Y-%m-%d")
    days_7 = datetime.timedelta(7)
    new_date = min_date - days_7
    new_date = new_date.strftime("%Y/%m/%d %H:%M:%S")
    new_date = new_date.replace("/", "-")

    max_date = datetime.datetime.strptime(max_date, "%Y-%m-%d")
    days_22 = datetime.timedelta(22)
    new_max_date = max_date + days_22
    new_max_date = new_max_date.strftime("%Y/%m/%d %H:%M:%S")
    new_max_date = new_max_date.replace("/", "-") 

    data = get_data(weir=weir, data_path=data_path)
    data.reset_index(inplace=True)
    data['INDEX'] = data['TIME'].copy()
    data = data.set_index('INDEX')
    # Gets lasts days but based on the custom risk_date
    data = data.loc[new_date: new_max_date]
    data["VERSCHIL"] = negative_backwater_to_zero(data["VERSCHIL"])

    df, df_veggie = get_predictions(min_date, max_date, weir, data_path, path_to_full_weather)
    counter = 1
    
    for index, row in df.iterrows():
        df_day = pd.DataFrame(row)
        last_day = datetime.datetime.strptime(index, "%Y-%m-%d")
        first_day = last_day - days_7
        new_dates= [last_day+datetime.timedelta(days=i) for i in range(0,21)]
        df_day = pd.DataFrame(new_dates)
        df_day.rename(columns={0:"time"}, inplace=True)
        df_day["values"] = row
        # print(df_day)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x= data["TIME"], y = data["VERSCHIL"]))#, row = counter, col = 1)
        # fig.add_trace(go.Scatter(x= data["TIME"], y = data["Q"]))#, row = counter, col = 1)
        fig.add_trace(go.Scatter(x =df_day["time"], y =df_day["values"]))#, row = counter, col = 1)
        fig.add_vline(x=last_day)#, row = counter, col = 1)
        fig.add_vline(x=first_day)#, row = counter, col = 1)
        fig.update_layout(title = "Day {} predictions".format(counter))
        counter += 1
        fig.show()
    return 

In [29]:
def plot_graphs_q_diveded_verschil(min_date, max_date, weir, data_path, path_to_full_weather = '../data/full_weather.csv'):
    min_date = datetime.datetime.strptime(min_date, "%Y-%m-%d")
    days_7 = datetime.timedelta(7)
    new_date = min_date - days_7
    new_date = new_date.strftime("%Y/%m/%d %H:%M:%S")
    new_date = new_date.replace("/", "-")

    max_date = datetime.datetime.strptime(max_date, "%Y-%m-%d")
    days_22 = datetime.timedelta(22)
    new_max_date = max_date + days_22
    new_max_date = new_max_date.strftime("%Y/%m/%d %H:%M:%S")
    new_max_date = new_max_date.replace("/", "-") 

    data = get_data(weir=weir, data_path=data_path)
    data.reset_index(inplace=True)
    data['INDEX'] = data['TIME'].copy()
    data = data.set_index('INDEX')
    # Gets lasts days but based on the custom risk_date
    data = data.loc[min_date: new_max_date]
    data["VERSCHIL"] = negative_backwater_to_zero(data["VERSCHIL"])

    df = get_predictions(min_date, max_date, weir, data_path, path_to_full_weather)
    df_deri = df[20]
    df_deri =df_deri.reset_index()
    df_deri['index'] = pd.to_datetime(df_deri["index"])
    df_deri["index"] = df_deri["index"] + days_22
    fig = go.Figure()
    
    data["q/verschil"] = np.where(data["Q"] != 0, data["Q"], data["VERSCHIL"]/data["Q"])
    fig.add_trace(go.Scatter(x= data["TIME"], y = data["q/verschil"]))#, row = counter, col = 1)
    # fig.add_trace(go.Scatter(x= data["TIME"], y = data["Q"]))#, row = counter, col = 1)
    fig.add_trace(go.Scatter(x = df_deri["index"], y = df_deri[20]))
    # fig.add_trace(go.Scatter(x =df_day["time"], y =df_day["values"]))#, row = counter, col = 1)
    # fig.add_vline(x=last_day)#, row = counter, col = 1)
    # fig.add_vline(x=first_day)#, row = counter, col = 1)
    # fig.update_layout(title = "Day {} predictions".format(counter))
    # counter += 1

    return fig.show()

In [42]:
plot_graphs_q_diveded_verschil('2020-7-1','2020-7-20',weir, data_path)

C:\Users\20193216\.conda\envs\DataScie\lib\site-packages\pandas\core\indexes\base.py:5277: FutureWarning:

Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.



                                               TIME    STUWVAK  VERSCHIL  \
INDEX                                                                      
2020-07-01 00:00:00+00:00 2020-07-01 00:00:00+00:00  211G_211F  0.056927   
2020-07-02 00:00:00+00:00 2020-07-02 00:00:00+00:00  211G_211F  0.045208   
2020-07-03 00:00:00+00:00 2020-07-03 00:00:00+00:00  211G_211F  0.041104   
2020-07-04 00:00:00+00:00 2020-07-04 00:00:00+00:00  211G_211F  0.040260   
2020-07-05 00:00:00+00:00 2020-07-05 00:00:00+00:00  211G_211F  0.043698   
2020-07-06 00:00:00+00:00 2020-07-06 00:00:00+00:00  211G_211F  0.047552   
2020-07-07 00:00:00+00:00 2020-07-07 00:00:00+00:00  211G_211F  0.045760   
2020-07-08 00:00:00+00:00 2020-07-08 00:00:00+00:00  211G_211F  0.045021   
2020-07-09 00:00:00+00:00 2020-07-09 00:00:00+00:00  211G_211F  0.048125   
2020-07-10 00:00:00+00:00 2020-07-10 00:00:00+00:00  211G_211F  0.053792   
2020-07-11 00:00:00+00:00 2020-07-11 00:00:00+00:00  211G_211F  0.053104   
2020-07-12 0

In [18]:
plot_graphs('2020-7-1','2020-7-20',weir, data_path)

In [30]:
weir = "211L_211K"
plot_graphs('2020-5-1','2020-5-20',weir, data_path)